In [17]:
!nvidia-smi

Thu Dec 30 05:43:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:01:00.0 Off |                    0 |
| N/A   44C    P0    61W / 275W |  19389MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:47:00.0 Off |                    0 |
| N/A   

In [2]:
!pip install torchio

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [14]:
!pip install monai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 709 kB 269 kB/s eta 0:00:01


In [3]:
import random
import os
import os.path
import numpy as np
import math
import tqdm
import random
import copy
import torchio

import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F 

import cv2
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter



In [11]:
t.backends.cudnn.enabled = True
os.environ["CUDA_VISIBLE_DEVICES"]="2"
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

#device = "cuda:0"
#device1 = "cuda:0"
#device2 = "cuda:0"

#train_path = "../../train_data_tcia.pth"

test_path = "Data/test_data_tcia.pth"

y_pred_path = "OutputsTcia/SABOSpred_tcia"
y_true_path = "OutputsTcia/SABOStrue_tcia"


In [5]:
# # Encoder - Decoder Network
class ChannelPool(nn.Module):
    def forward(self,x):
        return t.cat((t.max(x,1)[0].unsqueeze(1), t.mean(x,1).unsqueeze(1)),dim=1)

class SpatialGate(nn.Module):

    def __init__(self):
        super().__init__()
        self.compress = ChannelPool()
        self.conv = nn.Sequential(
            nn.Conv3d(2,1,kernel_size=7,stride=1,padding=3, bias=False),
            nn.BatchNorm3d(1,eps=1e-5,momentum=0.01,affine=True),
            nn.Sigmoid()
        )
    def forward(self,x):
        xcompress=self.compress(x)
        spatialAttention=self.conv(xcompress)
        return x*spatialAttention
    
class Flatten(nn.Module):
    def forward(self,x):
        return x.view(x.size(0),-1)

class ChannelGate(nn.Module):
  
    def __init__(self,channels,reductionRatio=16,poolTypes=['avg','max']):
        super().__init__()
        self.channels = channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(channels,channels//reductionRatio),
            nn.ReLU(),
            nn.Linear(channels//reductionRatio,channels)
        )
        self.poolTypes = poolTypes
  
    def forward(self,x):
        attentionSum = None
        for poolType in self.poolTypes:
            if poolType=='avg':
                avgPool = F.avg_pool3d(x,(x.size(2),x.size(3),x.size(4)),stride=(x.size(2),x.size(3),x.size(4)))
                channelAttention = self.mlp(avgPool)
            if poolType=='max':
                maxPool = F.max_pool3d(x,(x.size(2),x.size(3),x.size(4)),stride=(x.size(2),x.size(3),x.size(4)))
                channelAttention = self.mlp(maxPool)
        if attentionSum is None:
              attentionSum = channelAttention
        else:
              attentionSum=attentionSum+channelAttention
    
        scale=t.sigmoid(attentionSum).unsqueeze(2).unsqueeze(3).unsqueeze(4).expand_as(x)
        return x*scale


class CBAM(nn.Module):
  
    def __init__(self,channels,reductionRatio=16,poolTypes=['avg','max']):
        super().__init__()
        self.channelGate = ChannelGate(channels,reductionRatio,poolTypes)
        self.spatialGate = SpatialGate()

    def forward(self,x):
        x=self.spatialGate(x)
        x=self.channelGate(x)
        return x 
    
class CBAMResnetBlock(nn.Module):

    def __init__(self, in_channels, out_channels,kernelSize=3):
        super().__init__()
        self.resnetblock1 = nn.Sequential(
            nn.Conv3d(in_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=False)
            )
        self.resnetblock2 = nn.Sequential(
            nn.Conv3d(out_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=False),
            nn.Conv3d(out_channels,out_channels,kernel_size=kernelSize,padding=1),
            nn.BatchNorm3d(out_channels)
            )
        self.CBAM = CBAM(out_channels,16,['avg','max'])
        self.reluUnit = nn.Sequential(
            nn.ReLU(inplace=False)
            )

    def forward(self,x):
        x=self.resnetblock1(x)
        x1=x
        x=self.resnetblock2(x)
        x=self.CBAM(x)
        x=x+x1
        x=self.reluUnit(x)
        return x
    
class GSEncoder(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.downConv  = nn.Sequential(
            nn.Conv3d(1,32,kernel_size=3,stride=2,padding=1,bias=False),
            nn.BatchNorm3d(32),
            nn.LeakyReLU(inplace=False),
        )

        self.layer21 = nn.Sequential(
            CBAMResnetBlock(32,40),
            CBAMResnetBlock(40,40)
        )

        self.layer22 = nn.Sequential(
            CBAMResnetBlock(40,48),
            CBAMResnetBlock(48,48)
        )

        self.layer23 = CBAMResnetBlock(48,56)

    def forward(self,x):

        x0=self.downConv(x)
        x1=self.layer21(x0)
        x2=self.layer22(x1)
        x3=self.layer23(x2)
        #print(x3.shape)
        return x,x0,x1,x2,x3


class GSDecoder_part1(nn.Module):
    
    def __init__(self):
        super().__init__()
    
        self.layer23 = nn.Sequential(
            CBAMResnetBlock(56,56),
            CBAMResnetBlock(56,48)
        )
        self.layer22 = nn.Sequential(
            CBAMResnetBlock(96,48),
            CBAMResnetBlock(48,48),
            CBAMResnetBlock(48,40)
        )
        self.layer21 = nn.Sequential(
            CBAMResnetBlock(80,40),
            CBAMResnetBlock(40,40),
            CBAMResnetBlock(40,32)
        )
        self.layer20 = nn.Sequential(
            CBAMResnetBlock(64,32),
            CBAMResnetBlock(32,32)
        )
        self.transposeConv = nn.ConvTranspose3d(32,16,kernel_size=2,stride=2)
        self.layer10 = nn.Conv3d(17,16,kernel_size=3,padding=1)

    def forward(self,x,x0,x1,x2,x3):
        
        y=self.layer23(x3)
        y=t.cat([y,x2],dim=1)
        y=self.layer22(y)
        y=t.cat([y,x1],dim=1)
        y=self.layer21(y)
        y=t.cat([y,x0],dim=1)
        y=self.layer20(y)
        y=self.transposeConv(y)
        y=t.cat([y,x],dim=1)
        y=self.layer10(y)
        #print(y.shape)
        return y


class GSNetCore(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.encoder = GSEncoder()
        self.decoder_part1 = GSDecoder_part1()
  
    def forward(self,x):
        x,x0,x1,x2,x3=self.encoder(x)
        y=self.decoder_part1(x,x0,x1,x2,x3)
        return y


class GSNetModel(nn.Module):
  
    def __init__(self):
        super().__init__()
        self.core = GSNetCore()
        self.finalConv = nn.Sequential(
            nn.Conv3d(16,8,1),
            nn.Softmax(dim=1),
        ).cuda()
  
    def forward(self,x):
        y=self.core(x)
        y=self.finalConv(y)
        #print(y.shape)
        return y
    

In [6]:
class HaN_Dataset(Dataset):
    
    def __init__(self, root_dir=None, transform=False, alpha=1000, sigma=30, alpha_affine=0.04):
        super().__init__()
        self.path = root_dir
        self.datas = t.load(self.path)
        
        self.transform = transform
        self.alpha = alpha
        self.sigma = sigma
        self.alpha_affine = alpha_affine
    
    def __getitem__(self, index):
        data = self.datas[index]
        img = data['img'].numpy().astype(np.float32)
        
        if not self.transform:
            masklst = []
            for mask in data['mask']:
                if mask is None:
                    mask = np.zeros((1,img.shape[0],img.shape[1],img.shape[2])).astype(np.uint8)
                masklst.append(mask.astype(np.uint8).reshape((1,img.shape[0],img.shape[1],img.shape[2]))) 
            mask0 = np.zeros_like(masklst[0]).astype(np.uint8)
            for mask in masklst:
                mask0 = np.logical_or(mask0, mask).astype(np.uint8)
            mask0 = 1 - mask0
            return t.from_numpy(img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))), t.from_numpy(np.concatenate([mask0]+masklst, axis=0)), True
        
       
        im_merge = np.concatenate([img[...,None]]+[mask.astype(np.float32)[...,None] for mask in data['mask']], axis=3)
        # Apply transformation on image
        im_merge_t, new_img = self.elastic_transform3Dv2(im_merge,self.alpha,self.sigma,min(im_merge.shape[1:-1])*self.alpha_affine)
        # Split image and mask ::2, ::2, ::2
        im_t = im_merge_t[...,0]
        im_mask_t = im_merge_t[..., 1:].astype(np.uint8).transpose(3, 0, 1, 2)
        mask0 = np.zeros_like(im_mask_t[0, :, :, :]).reshape((1,)+im_mask_t.shape[1:]).astype(np.uint8)
        im_mask_t_lst = []
        flagvect = np.ones((8,), np.float32)
        retflag = True
        for i in range(7):
            im_mask_t_lst.append(im_mask_t[i,:,:,:].reshape((1,)+im_mask_t.shape[1:]))
            if im_mask_t[i,:,:,:].max() != 1: 
                retflag = False
                flagvect[i+1] = 0
            mask0 = np.logical_or(mask0, im_mask_t[i,:,:,:]).astype(np.uint8)
        if not retflag: flagvect[0] = 0
        mask0 = 1 - mask0
        return t.from_numpy(im_t.reshape((1,)+im_t.shape[:3])), t.from_numpy(np.concatenate([mask0]+im_mask_t_lst, axis=0)), flagvect
        
    def __len__(self):
        return len(self.datas)
    
    def elastic_transform3Dv2(self, image, alpha, sigma, alpha_affine, random_state=None):
        """Elastic deformation of images as described in [Simard2003]_ (with modifications).
        .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
             Convolutional Neural Networks applied to Visual Document Analysis", in
             Proc. of the International Conference on Document Analysis and
             Recognition, 2003.
         Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
         From https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
        """
        # affine and deformation must be slice by slice and fixed for slices
        if random_state is None:
            random_state = np.random.RandomState(None)
        shape = image.shape # image is contatenated, the first channel [:,:,:,0] is the image, the second channel 
        # [:,:,:,1] is the mask. The two channel are under the same tranformation.
        shape_size = shape[:-1] # z y x
        # Random affine
        shape_size_aff = shape[1:-1] # y x
        center_square = np.float32(shape_size_aff) // 2
        square_size = min(shape_size_aff) // 3
        pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
        pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
        M = cv2.getAffineTransform(pts1, pts2)
        new_img = np.zeros_like(image)
        for i in range(shape[0]):
            new_img[i,:,:,0] = cv2.warpAffine(image[i,:,:,0], M, shape_size_aff[::-1], borderMode=cv2.BORDER_CONSTANT, borderValue=0.)
            for j in range(1, 8):
                new_img[i,:,:,j] = cv2.warpAffine(image[i,:,:,j], M, shape_size_aff[::-1], flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_TRANSPARENT, borderValue=0)
        dx = gaussian_filter((random_state.rand(*shape[1:-1]) * 2 - 1), sigma) * alpha
        dy = gaussian_filter((random_state.rand(*shape[1:-1]) * 2 - 1), sigma) * alpha
        x, y = np.meshgrid(np.arange(shape_size_aff[1]), np.arange(shape_size_aff[0]))
        indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
        new_img2 = np.zeros_like(image)
        for i in range(shape[0]):
            new_img2[i,:,:,0] = map_coordinates(new_img[i,:,:,0], indices, order=1, mode='constant').reshape(shape[1:-1])
            for j in range(1, 8):
                new_img2[i,:,:,j] = map_coordinates(new_img[i,:,:,j], indices, order=0, mode='constant').reshape(shape[1:-1])
        return np.array(new_img2), new_img
      
# %%


In [7]:
def get_yPredDash(y_pred):
    
    y_pred = y_pred.data.cpu().numpy() # inference should be arg max
    y_pred_1 = np.zeros_like(y_pred).astype(np.uint8)
    y_pred = np.argmax(y_pred, axis=1).squeeze() # z y x
    
    for i in range(8):
        y_pred_1[:,i,:,:,:] = y_pred==i
    
    return y_pred_1


In [8]:
# 1. TCIA test data loading
test_path = "Data/test_data_tcia.pth"
testdataset = HaN_Dataset(test_path, transform=False)
testdataloader = DataLoader(testdataset, batch_size=1)
print(len(testdataloader))
weight_path = "Weights/SABOSNet_tcia.pth"
#dsc_path = "EvaluationMetrics/Tcia_20_DSCNew.npy"
hd_path= "EvaluationMetrics/Tcia_20_HDNew.npy"
msd_path = "EvaluationMetrics/Tcia_20_MSDNew.npy"
sen_path = "EvaluationMetrics/Tcia_20_SENNew.npy"
ppv_path = "EvaluationMetrics/Tcia_20_PPVNew.npy"

20


In [15]:
# 2. New Seenia Evaluation metrics TCIA
from monai.metrics import HausdorffDistanceMetric,SurfaceDistanceMetric
hd = HausdorffDistanceMetric(percentile=95,reduction="none",include_background=False)
msd = SurfaceDistanceMetric(include_background=True)
def compute_performance(pred, gt):
    #print("pred shape : ", pred.shape, "Ground shape :" , gt.shape)
    pred=(pred>=0.5).type(t.float)
    pred = pred.data.cpu()
    gt = gt.data.cpu()
    sen=[]
    ppv=[]
    for i in range(1,8):
        pred1=pred[0][i]
        gt1=gt[0][i]
        TP = (gt1 * pred1).sum()
        TN = ((1-gt1) * (1-pred1)).sum()
        FP = ((1-gt1) * (pred1)).sum()
        FN = ((gt1) * (1-pred1)).sum()
        sen.append(TP/(TP+FN))
        ppv.append(TP/(TP+FP))
    sen=np.array(sen)
    ppv=np.array(ppv)
         
    hds=hd(gt[:,1:,...].permute(0,1,3,4,2),pred[:,1:,...].permute(0,1,3,4,2))
    msds=msd(gt[:,1:,...].permute(0,1,3,4,2),pred[:,1:,...].permute(0,1,3,4,2))
    
    hds=np.array(hds)
    msds=np.array(msds)
    return hds,msds,sen,ppv


In [16]:
# 3. Evaluation TCIA
import time
SABOSNet_Model = GSNetModel().cuda()
loaded_model = t.load(weight_path, map_location='cuda:0')
SABOSNet_Model.load_state_dict(loaded_model["model_state_dict"])
print("weight loaded")
SABOSNet_Model.eval()
print("Ready for evaluation")
hd = HausdorffDistanceMetric(percentile=95,reduction="none",include_background=True)
testtq = tqdm.tqdm(testdataloader)#, desc='loss', leave=True)
predtime=[]

msdsL=[]
hdsL=[]
senL=[]
ppvL=[]
i=0
for x_test, y_test, _ in testtq:
        with t.no_grad():
            t1=time.time()
            x_test = x_test.cuda()
            y_test = y_test.cuda()
            o = SABOSNet_Model(x_test)
            ti=round(time.time()-t1 , 3)
            predtime.append(ti)
            y_predDash = get_yPredDash(o)
            #np.save(y_pred_path + str(i) + ".npy",y_predDash)
            ti=round(time.time()-t1 , 3)
            #np.save(y_true_path + str(i) + ".npy",y_test.cpu())
            o = o.cuda()
            hds, msds, sen, ppv = compute_performance(o, y_test)
            senL.append(sen)
            ppvL.append(ppv)
            hdsL.append(hds[0])
            msdsL.append(msds[0])
            i=i+1
            del x_test, y_test, o
        
hdsL=np.array(hdsL)
msdsL=np.array(msdsL)
senL=np.array(senL)
ppvL=np.array(ppvL)

print("HD New: ", hdsL.mean(axis=0),"+/-", np.std(hdsL, axis=0))
print("MSD New : ", msdsL.mean(axis=0),"+/-", np.std(msdsL, axis=0))
print("Sensitivity New : ", senL.mean(axis=0), "+/-", np.std(senL, axis=0))
print("Positive Predictive Value-PPV New : ", ppvL.mean(axis=0), "+/-", np.std(ppvL, axis=0))
print ("Average time : ", np.mean(predtime))
            
np.save(hd_path, hdsL)
np.save(msd_path, msdsL)
np.save(sen_path, senL)
np.save(ppv_path, ppvL)

weight loaded
Ready for evaluation


100%|███████████████████████████████████████████| 20/20 [00:50<00:00,  2.55s/it]

HD New:  [2.90798554 3.23503585 1.35604779 1.55103791 1.70838309 3.95297094
 3.03074054] +/- [1.02746925 1.28903679 0.51101309 0.70285325 1.33091659 2.12697456
 0.78397179]
MSD New :  [1.04009279 0.9492215  0.40381043 0.4740463  0.43876314 1.22455971
 1.11603386] +/- [0.33375619 0.4948407  0.13492341 0.23415994 0.17470953 0.31121918
 0.18016656]
Sensitivity New :  [0.8835704  0.65535367 0.9438397  0.7554432  0.7742028  0.89493835
 0.9005798 ] +/- [0.06390476 0.1573351  0.03244483 0.15533407 0.06641252 0.08176685
 0.05789993]
Positive Predictive Value-PPV New :  [0.9010743  0.65017354 0.9201522  0.7320004  0.7602762  0.8287237
 0.84882754] +/- [0.04027881 0.14747618 0.0337702  0.11709588 0.10157115 0.09763792
 0.07081824]
Average time :  0.024100000000000003
